<a href="https://colab.research.google.com/github/pphos/rock_paper_scissors/blob/master/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>